In [ ]:
# import libraries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow
import pickle

In [7]:
# load the trained model ,scaler pickle, one hot
model = tensorflow.keras.models.load_model('model.h5')
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)
with open('label_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)
